In [1]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import img_to_array, array_to_img, img_to_array, load_img
import os
import shutil

In [6]:
ORIGINAL_DATA_PATH = './data/images'
AUGMENTED_DATA_PATH = './data/augmented_images'

TRAIN_DIRECTORY = 'train'

SUB_DIRECTORIES = [TRAIN_DIRECTORY]
DATA_DIRECTORIES = ['Chickenpox', 'Cowpox', 'Healthy', 'HFMD', 'Measles', 'Monkeypox']

DS_STORE = '.DS_Store'

ROTATION_RANGE = 90
WIDTH_SHIFT_RANGE = 0.2
HEIGHT_SHIFT_RANGE = 0.2
SHEAR_RANGE = 0.2
ZOOM_RANGE = 0.2
HORIZONTAL_FLIP = True
FILL_MODE = 'nearest'

In [3]:
datagen = ImageDataGenerator(
    rotation_range=ROTATION_RANGE,
    width_shift_range=WIDTH_SHIFT_RANGE,
    height_shift_range=HEIGHT_SHIFT_RANGE,
    shear_range=SHEAR_RANGE,
    zoom_range=ZOOM_RANGE,
    horizontal_flip=HORIZONTAL_FLIP,
    fill_mode=FILL_MODE)

In [5]:
for sub_directory in SUB_DIRECTORIES:
    path = AUGMENTED_DATA_PATH
    path = os.path.join(path, sub_directory)
    os.mkdir(path)

for data_directory in DATA_DIRECTORIES:
    for sub_directory in SUB_DIRECTORIES:
        path = AUGMENTED_DATA_PATH
        path = os.path.join(path, sub_directory, data_directory)
        os.mkdir(path)

In [7]:
for data_directory in DATA_DIRECTORIES:
    for sub_directory in SUB_DIRECTORIES:
        path_o = os.path.join(ORIGINAL_DATA_PATH, sub_directory, data_directory)
        path_a = os.path.join(AUGMENTED_DATA_PATH, sub_directory, data_directory)
        files = os.listdir(path_o)
        for index, file in enumerate(files):
            if DS_STORE in file: ### Skip .DS_Store file
                continue
            print(f'{path_o}/{file}') ### Print the pathname and file name to get an idea about the processing file
            img = load_img(str(os.path.join(path_o, file))) ### This is a PIL image
            x = img_to_array(img) ### This is a NumPy array with shape (3, 150, 150)
            x = x.reshape((1,) + x.shape) ### This is a NumPy array with shape (1, 3, 150, 150)
            ### The .flow() command generates batches of randomly transformed images and save the results in the ./data/augmented/<sub_directory>/<data_directory> along with the original image
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=path_a, save_prefix=file.split(".")[0], save_format='jpg'):
                i += 1
                if i > 5:
                    break ### Stop looping the generator infinitely
            ### Save the original image in ./data/augmented/<sub_directory>/<data_directory> as well
            shutil.copy(os.path.join(path_o, file), path_a)   

./data/images/train/Chickenpox/CHP_05_01.jpg
./data/images/train/Chickenpox/CHP_17_01.jpg
./data/images/train/Chickenpox/CHP_09_01.jpg
./data/images/train/Chickenpox/CHP_21_01.jpg
./data/images/train/Chickenpox/CHP_42_01.jpg
./data/images/train/Chickenpox/CHP_33_01.jpg
./data/images/train/Chickenpox/CHP_50_01.jpg
./data/images/train/Chickenpox/CHP_15_01.jpg
./data/images/train/Chickenpox/CHP_19_01.jpg
./data/images/train/Chickenpox/CHP_07_01.jpg
./data/images/train/Chickenpox/CHP_52_01.jpg
./data/images/train/Chickenpox/CHP_31_01.jpg
./data/images/train/Chickenpox/CHP_40_01.jpg
./data/images/train/Chickenpox/CHP_23_01.jpg
./data/images/train/Chickenpox/CHP_15_02.jpg
./data/images/train/Chickenpox/CHP_52_02.jpg
./data/images/train/Chickenpox/CHP_44_01.jpg
./data/images/train/Chickenpox/CHP_39_01.jpg
./data/images/train/Chickenpox/CHP_27_01.jpg
./data/images/train/Chickenpox/CHP_56_01.jpg
./data/images/train/Chickenpox/CHP_48_01.jpg
./data/images/train/Chickenpox/CHP_35_01.jpg
./data/ima